# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# Wcode Class

In [114]:
class Wcode < Array
  def initialize(word = Group::Identity)
    @words = [word]
    #---
    if word != Group::Identity
      arr = Array.new
      word.flatten.factors.each do |f|
        arr += [Rseq.index(f)+1, Rseq.index(f.inverse)+1]
      end
      arr << arr.shift
      arr.each_slice(2){|a| self << a}
    end
  end
  attr_reader :words
  #---
  def size(); self.flatten.size end
  def max(); self.flatten.max end
  def copy()
    wc = self.class.new
    wc.each_with_index{|a, i| wc[i] = self[i]}
    wc
  end
  #---
  def flatten()
    myarr = Array.new
    self.each{|a| myarr += a}
    myarr
  end
  #---
  def to_word()
    flt = self.flatten
    flt.unshift(flt.pop)
    gens = []
    flt.each_slice(2){|s| gens << Rseq[s[0]-1] }
    Word.new(gens)
  end
  #---
  def product(another)
    pop = self[-1].last
    self[-1][1] = another[-1][1]
    another[-1][1] = pop
    self + another
  end
  #---
  def devide(idxs)
    facs = []
    ### TODO ###
    return facs
  end
  #---
  ## Turaev cobracket for Wcodes
  def cobracket()
    odd_pairs=(1..self.size/2).to_a.map{|x| 2*x-1}.combination(2)
    factors  = odd_pairs.map do |odds|
      order = self.get_four_tuple(odds)
      factors = self.devide(odds).map(&:narrow)
      #---
      {order: order, linking: order.linking, factors: factors }
    end
    return factors.select{|prd| prd[:linking] != 0}
  end  
end; nil

#-----------------------------------
class FourTuple < Array
  def initialize(arr, mod)
    @modulo = mod
    4.times{|i| self[i] = arr[i]}
    @odd_nums = self.select{|n| n.odd?}.sort
  end
  
  def linking()
    odd1 = @odd_nums[0]
    if (self.index(odd1) - self.index(odd1+1) + 1).even?
      # [o,e,*,*] / [o,*,*,e] / [*,o,e,*] / [*,*,o,e] (and reverse 'o' and 'e')
      return 0
    else
      # [o1,e2,e1,o2] --> linking = +1 and [o2,o1,e2,e1] --> linking = +1
      # The sign of linking is change by (o1<->e1) and (e2<->o2)
      idxs = [self.index(odd1), self.index(odd1+1)] 
      sign0 = (idxs[0] < idxs[1]) ? 1 : -1
      rng = idxs.sort
      sign2 = self[rng[0]..rng[1]][1].odd? ? 1 : -1
      #---
      sign = sign0*sign2
    end
  end
end;nil

In [115]:
w = Word.new(a[1], b[2], b[1].inverse)
p w.show

wc = Wcode.new(w)
p wc, wc.flatten, wc.to_word.show
nil

"adB"
[[3, 8], [6, 2], [4, 1]]
[3, 8, 6, 2, 4, 1]
"adB"


In [116]:
w2 = Word.new(a[2])
wc2 = Wcode.new(w2)

p wc.to_s + '*' + wc2.to_s

p wc.product(wc2)
nil

"[[3, 8], [6, 2], [4, 1]]*[[7, 5]]"
[[3, 8], [6, 2], [4, 5], [7, 1]]


In [ ]:
wc = wc_arr.inject(:product)
p wc.to_word.show, wc.show
printf("===\n")

wc.cobracket.each do |h|
  p "order : #{h[:order]}"
  p "linking : #{h[:linking]}"
  p "factors : " + h[:factors].map(&:show).join(' , ')
  prd = h[:factors].inject(&:product)
  p prd.show
  p h[:factors].map(&:to_word).map(&:show).join(" \u{2227} ")
  p h[:factors].map(&:to_word).map(&:cyclic_reduce).map(&:show).join(" \u{2227} ")
#  p prd.to_word.show
  printf("-----\n")
end
nil

## Turaev cobraket

In [ ]:
def cobracket(simple_words_arr)
  wa = simple_words_arr
  code = Wcode.new(wa)
  #---
  cob = code.cobracket
  
  unless cob.empty?
    cob.each do |h|
      h[:words] = h[:factors].map do |f|
        evens = f.values.flatten.select{|v| v.even?}.sort
        evens.map{|e| wa[e/2]}
      end
    end
  end

  return cob
end;nil

# Execution

In [ ]:
a1, a2 = Word.new(a[1]), Word.new(a[2])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten
samples = [
  [a1],
  [a1, Word.new(b[2])],
  [a1, Word.new(b[2]).inverse],
  #---
  (1..4).map{|i| a1},
  #---
  [a1, (comms[2].inverse)],
  [a1, comms[2]],
  [a1] + (1..3).map{|i| comms[2]},
  [a1] + (1..3).map{|i| a2},
  [a1] + (1..3).map{|i| Word.new(a[2], b[2].inverse)},
  [a1, comms[2], c, comms[2], c.inverse],
  #---
  [a1, Group.commutator(a[2], a[3])],
  [a1, comms[2]*comms[3], Group.commutator(a[2], a[3]), (comms[2]*comms[3]).inverse]
  ]

samples.each do |wa|
  ws = wa.inject(:*).show.gsub(/[()]+/, '.').gsub(/^[.]|[.]$/, '')
  terms = cobracket(wa).map do |h|
    coeff = (h[:linking]==1) ? '' : '(-1)'
    coeff + h[:words].map{|a| a.inject(:*).cyclic_reduce.show }.join(" \u{2227} ")
  end
  p "#{ws}  |--\u{03B4}--> #{terms.join(' + ')}"
end;nil

"a  |--δ-->  0"
"a.d  |--δ-->  a ∧ d"
"a.D  |--δ-->  0"
"a.a.a.a  |--δ-->  (-1) aa ∧ aa"
"abA  |--δ-->  (-1) 1 ∧ b"
"abAB  |--δ-->  0"
"a.dcDC  |--δ-->  a ∧ dcDC"
"a.cdCD  |--δ-->  0"
"a.cdCD.cdCD.cdCD  |--δ-->  (-1) acdCDcdCD ∧ cdCD + (-1) acdCD ∧ cdCDcdCD"
"a.c.c.c  |--δ-->  (-1) acc ∧ c + (-1) ac ∧ cc"
"a.cD.cD.cD  |--δ-->  (-1) acDcD ∧ cD + (-1) acD ∧ cDcD"
"a.cdCD.abA.cdCD.aBA  |--δ-->  (-1) acdCD ∧ cdCD + (-1) cdCD ∧ cdCDa"
"a.ceCE  |--δ-->  aceC ∧ E + (-1) ace ∧ CE + ac ∧ C"
"a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  acdCDefEFceCfeFEdcDC ∧ E + (-1) acdCDefEFcefeFEdcDC ∧ CE + acdCDefEFcfeFEdcDC ∧ C"

In [ ]:
warr = samples[-3]
wc = Wcode.new(warr)
p wc.show, wc.words.map(&:show)
nil

In [ ]:
cob = cobracket(warr)
cob.each do |h|
  p "order: #{h[:order]}"
  p "linking: #{h[:linking]}"
  p "factors: #{h[:factors].map(&:show)}"
  p "words: #{h[:words].map{|a| a.inject(:*).show}}"
  p h[:words].map{|a| a.inject(:*).cyclic_reduce.show }.join(" \u{2227} ")
  printf("-- -- --\n")
end
nil

### TODO
$[a_{1},b_{1}]$ の処理において、
* [x] [5, 0, 6, 7] の sign が正しくないようだ。
* [x] a(x)A [ $a_{1}\otimes a_{1}^{-1}$ ] の集計もおかしい。
* [x] $a_{1}^{3}$ の計算がエラーになる。
* [ ] simple subword たちによる計算を実装する。